In [43]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip()
        tickers.append(ticker)
    return sorted(tickers)

sp500_tickers = get_sp500_tickers()
print(sp500_tickers)

['A', 'AAPL', 'ABBV', 'ABNB', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APO', 'APTV', 'ARE', 'ATO', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXON', 'AXP', 'AZO', 'BA', 'BAC', 'BALL', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BG', 'BIIB', 'BK', 'BKNG', 'BKR', 'BLDR', 'BLK', 'BMY', 'BR', 'BRK.B', 'BRO', 'BSX', 'BWA', 'BX', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CEG', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP', 'COR', 'COST', 'CPAY', 'CPB', 'CPRT', 'CPT', 'CRL', 'CRM', 'CRWD', 'CSCO', 'CSGP', 'CSX', 'CTAS', 'CTRA', 'CTSH', 'CTVA', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DAY', 'DD', 'DE', 'DECK', 'DELL', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DLR',

In [44]:
import yfinance as yf


def get_stock_data(ticker, period='1mo'):
    """
    Fetches the closing stock data for a given ticker symbol and period.

    Parameters:
    ticker (str): The ticker symbol of the stock.
    period (str): The period over which to fetch the data. Default is '1mo'.

    Returns:
    pandas.DataFrame: A dataframe containing the closing stock data.
    """
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period).rename(columns=lambda x: x.lower())
    return hist


# Example usage
closing_data = get_stock_data('AAPL', '3mo')['close']
print(closing_data)

Date
2024-11-14 00:00:00-05:00    227.969360
2024-11-15 00:00:00-05:00    224.752884
2024-11-18 00:00:00-05:00    227.769577
2024-11-19 00:00:00-05:00    228.029282
2024-11-20 00:00:00-05:00    228.748489
                                ...    
2025-02-07 00:00:00-05:00    227.380005
2025-02-10 00:00:00-05:00    227.649994
2025-02-11 00:00:00-05:00    232.619995
2025-02-12 00:00:00-05:00    236.869995
2025-02-13 00:00:00-05:00    241.529999
Name: close, Length: 61, dtype: float64


In [45]:
def get_next_earnings_date(ticker):
    """
    Fetches the next earnings report date for a given ticker symbol.

    Parameters:
    ticker (str): The ticker symbol of the stock.

    Returns:
    str: The next earnings report date.
    """
    stock = yf.Ticker(ticker)
    earnings_date = stock.earnings_dates.index[0]
    return earnings_date

# Example usage
next_earnings_date = get_next_earnings_date('AAPL')
print(next_earnings_date)

2026-01-28 16:00:00-05:00


In [90]:
def get_sorted_price_changes(tickers, period='1mo', ascending=False):
    """
    Returns a sorted pandas DataFrame containing the sorted change percentage for a given list of stocks over a given period.

    Parameters:
    tickers (list): List of ticker symbols.
    period (str): The period over which to fetch the data. Default is '1mo'.

    Returns:
    pandas.DataFrame: A sorted dataframe containing the ticker symbols and their corresponding price change percentages.
    """
    price_changes = []

    for ticker in tickers:
        try:
            closing_data = get_stock_data(ticker, period)['close']
            if not closing_data.empty:
                price_change = ((closing_data.iloc[-1] - closing_data.iloc[0]) / closing_data.iloc[0]) * 100
                price_changes.append({'Ticker': ticker, 'Price Change (%)': price_change})
            else:
                tickers.remove(ticker)
                print(f"No data available for {ticker}")
        except Exception as e:
            tickers.remove(ticker)
            print(f"Error fetching data for {ticker}: {e}")

    price_changes_df = pd.DataFrame(price_changes)
    sorted_price_changes_df = price_changes_df.sort_values(by='Price Change (%)', ascending=ascending).reset_index(drop=True)
    
    return sorted_price_changes_df

PERIOD = '2wk'
# Example usage
sorted_price_changes = get_sorted_price_changes(sp500_tickers, PERIOD, True)
print(sorted_price_changes)

    Ticker  Price Change (%)
0      WST        -41.666926
1      FMC        -34.187881
2     SWKS        -25.439389
3     MRNA        -19.025871
4      HII        -18.473075
..     ...               ...
493   EXPE         18.865166
494   UBER         20.104716
495   INTC         24.189392
496   PLTR         42.938546
497   SMCI         48.246838

[498 rows x 2 columns]


In [91]:
# take the dataframe and obtain the subset of tickers within a given change 

subset_df = sorted_price_changes[(sorted_price_changes['Price Change (%)'] <= -5)]

print(subset_df)


   Ticker  Price Change (%)
0     WST        -41.666926
1     FMC        -34.187881
2    SWKS        -25.439389
3    MRNA        -19.025871
4     HII        -18.473075
..    ...               ...
82    DFS         -5.460240
83    PWR         -5.445207
84    MOS         -5.234848
85    WBA         -5.155640
86    BMY         -5.055131

[87 rows x 2 columns]


In [ ]:
from stock_indicators import indicators, Quote

oversold_ticker = subset_df['Ticker'].iloc[0]
print(f'Oversold Stock {oversold_ticker}')
data = get_stock_data(oversold_ticker, PERIOD)
# atr = ATR(data.high, data.low, data.close, timeperiod=2).dropna()

# Read and print the stock tickers that make up S&P500

# print(tickers.head())
quotes_list = [
    Quote(d,o,h,l,c,v) 
    for d,o,h,l,c,v 
    in zip(data.index, data['open'], data['high'], data['low'], data['close'], data['volume'])
]

supertrend = indicators.get_super_trend(quotes_list, 10, 3)
uppertrend = []
lowertrend = []
for r in supertrend:
    lowertrend.append(r.upper_band)
    uppertrend.append(r.lower_band)
    
print(lowertrend)

Oversold Stock DAL
[None, None, None, None, None, None, None, None, None, None]


In [87]:
def get_analyst_ratings(ticker):
    """
    Fetches the analysts' ratings and price targets for a given ticker symbol.

    Parameters:
    ticker (str): The ticker symbol of the stock.

    Returns:
    pandas.DataFrame: A dataframe containing the analysts' ratings and price targets.
    """

    rating = {
        'strongBuy': 5,
        'buy': 4,
        'hold': 3,
        'sell': 2,
        'strongSell': 1
    }
    
    total = 0
    stock = yf.Ticker(ticker)
    analyst_info = stock.recommendations
    analyst_info['quant'] = analyst_info.apply(lambda row: sum([row[r]*rating[r] for r in rating])/row[rating.keys()].sum(), axis=1)
    return analyst_info


# Example usage
ticker = oversold_ticker
analyst_ratings = get_analyst_ratings(ticker)

print(analyst_ratings)

  period  strongBuy  buy  hold  sell  strongSell     quant
0     0m          6   16     0     0           1  4.130435
1    -1m          6   16     0     0           1  4.130435
2    -2m          6   16     0     0           1  4.130435
3    -3m          6   16     0     0           1  4.130435


In [88]:
def get_price_targets(ticker):
    """
    Fetches the price targets for a given ticker symbol.

    Parameters:
    ticker (str): The ticker symbol of the stock.

    Returns:
    pandas.DataFrame: A dataframe containing the price targets.
    """
    stock = yf.Ticker(ticker)
    price_targets = stock.analyst_price_targets
    return price_targets

# Example usage
ticker = oversold_ticker
price_targets = get_price_targets(ticker)
print(price_targets)

{'current': 64.06, 'high': 100.0, 'low': 44.0, 'mean': 79.92864, 'median': 80.0}


In [93]:
def get_pe_ratio(ticker):
    """
    Fetches the PE ratio for a given ticker symbol.

    Parameters:
    ticker (str): The ticker symbol of the stock.

    Returns:
    float: The PE ratio of the stock.
    """
    stock = yf.Ticker(ticker)
    pe_ratio = stock.info.get('trailingPE', None)
    return pe_ratio

# Example usage
ticker = oversold_ticker
pe_ratio = get_pe_ratio(ticker)
print(f"The PE ratio of {ticker} is {pe_ratio}")

The PE ratio of DAL is 12.018762
